In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.11.7


In [3]:
import pickle
import pandas as pd

In [4]:
YEAR = 2023
MONTH = 3

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{YEAR:04d}-{MONTH:02d}.parquet')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

### Q1. What's the standard deviation of the predicted duration for this dataset?

In [9]:
y_pred.std()

6.247488852238703

### Q2. Preparing the output
First, let's create an artificial ride_id column:

In [10]:
df['ride_id'] = f'{YEAR:04d}/{MONTH:02d}_' + df.index.astype('str')

Next, write the ride id and the predictions to a dataframe with results.

In [14]:
df_predictions = df[['ride_id']].copy()
df_predictions['predictions'] = y_pred
df_predictions

,ride_id,predictions
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486
...,...,...
3403761,2023/03_3403761,11.952635
3403762,2023/03_3403762,20.049958
3403763,2023/03_3403763,11.595336
3403764,2023/03_3403764,13.113178



Save it as parquet:

In [16]:
output_file = f'predictions_{YEAR:04d}-{MONTH:02d}.parquet'
output_file

'predictions_2023-03.parquet'

In [17]:
df_predictions.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [18]:
!ls -lh 

total 66M
-rw-rw-r-- 1 mlops_zoomcamp_gcp mlops_zoomcamp_gcp  267 Jun 15 09:23 Pipfile
-rw-r--r-- 1 mlops_zoomcamp_gcp mlops_zoomcamp_gcp 146K Jun 15 09:23 Pipfile.lock
-rw-rw-r-- 1 mlops_zoomcamp_gcp mlops_zoomcamp_gcp  17K Jun 15 09:00 model.bin
-rw-rw-r-- 1 mlops_zoomcamp_gcp mlops_zoomcamp_gcp  66M Jun 16 13:22 predictions_2023-03.parquet
-rw-rw-r-- 1 mlops_zoomcamp_gcp mlops_zoomcamp_gcp 4.4K Jun 15 10:15 starter.ipynb
